# The usage of full-saddle-point dipoles

## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import os
import shutil
import h5py
import sys
import subprocess
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

import XUV_refractive_index as XUV_index

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

## Load data

We load the data from the pulse propagation in the Pythonic data container. It contains the data about the pulse propagation and some firther characteristics. The data from the micrscopic response and harmonic signal will be loaded later (these data are large, and we will need only a part of them according to the chosen analyses.)

In [ ]:
# series_version = 'v3'
# h5file1 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'Bessel', series_version, 'results_Bessel_7.h5')
# h5file2 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'Bessel', series_version, 'results_Bessel_9.h5')

h5file1 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'coherence_map', 't1', 'results_map1.h5')
# h5file1 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'coherence_map_short', 't2', 'results_map2.h5')
# h5file2 = os.path.join('/mnt','d','sharepoint', 'OneDrive - ELI Beamlines', 'data', 'Sunrise', 'demos', 'Bessel', 'v4', 'results_Bessel_7.h5')



# simulation = 'bigpf'
# h5file1 = os.path.join('/mnt','d', 'data', 'Sunrise', simulation, 'results.h5')
# h5file2 = os.path.join('/mnt','d', 'data', 'Sunrise', simulation, 'results_Hankel.h5')

# with h5py.File(h5file1,'r') as f, h5py.File(h5file2,'r') as f2:

#     # load cuprad data = pulse propagation
#     CUPRAD_res = dfC.get_data(f)
#     CUPRAD_res2 = dfC.get_data(f2)
#     # CUPRAD_res.get_plasma(f)
r_full_resolution = [False, 10e-6/5., 180e-6]
with h5py.File(h5file1,'r') as f:

    # load cuprad data = pulse propagation
    CUPRAD_res = dfC.get_data(f,r_resolution=r_full_resolution)
    CUPRAD_res.get_plasma(f,r_resolution=r_full_resolution)
    CUPRAD_res.vacuum_shift()
    CUPRAD_res.complexify_envel(output='add')

    Intensity_map = mn.FieldToIntensitySI(abs(CUPRAD_res.E_zrt_cmplx_envel))
    phase_map = np.angle(CUPRAD_res.E_zrt_cmplx_envel)

    gas = mn.readscalardataset(f,MMA.paths['global_inputs']+'/gas_preset', 'S')
    # CUPRAD_res2 = dfC.get_data(f2)
    # CUPRAD_res.get_plasma(f)

Ip_au = HHG.Ip_list[gas]   
omega0_au = mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau')

In [ ]:
Intensity_spatial_gradients = np.gradient(Intensity_map,CUPRAD_res.zgrid,axis=0,edge_order=2)



phase_spatial_gradients = np.zeros(np.shape(phase_map))
for k1 in range(CUPRAD_res.Nr):
    # print(k1)
    for k2 in range(CUPRAD_res.Nt):
        # phase_rfix = np.unwrap(phase_map[:,k1,k2])
        # phase_spatial_gradients[:,k1,k2] = np.gradient(phase_rfix,CUPRAD_res.zgrid,edge_order=2)
        phase_spatial_gradients[:,k1,k2] = np.gradient(np.unwrap(phase_map[:,k1,k2]),CUPRAD_res.zgrid,edge_order=2)

In [ ]:
image = pp.figure_driver()    
image.sf = [pp.plotter() for k2 in range(16)]


image.sf[0].args = [CUPRAD_res.zgrid, phase_spatial_gradients[:,0,CUPRAD_res.Nt//2]]

pp.plot_preset(image)

In [ ]:
Horder = 17

dispersion_tables = 'NIST'

nXUV = XUV_index.nXUV(Horder*CUPRAD_res.omega0, gas+'_'+dispersion_tables, 1e-3*CUPRAD_res.pressure_mbar, complex=False)
print(CUPRAD_res.pressure_mbar)
print(XUV_index.susc_ref(Horder*CUPRAD_res.omega0,  gas+'_'+dispersion_tables), nXUV, np.sqrt(1.+1e-3*CUPRAD_res.pressure_mbar*XUV_index.susc_ref(Horder*CUPRAD_res.omega0,  gas+'_'+dispersion_tables)))
print(CUPRAD_res.k0_wave*(nXUV-1.))

In [ ]:
# Code to generate the figure of plasma channel
rlim = 150

fig = plt.figure(figsize=(14, 5.5))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((1, 2), (0, 0))  # Upper left
ax2 = plt.subplot2grid((1, 2), (0, 1))  # Upper right


symmetric_y, symmetric_data =  mn.symmetrize_y(CUPRAD_res.plasma.rgrid,CUPRAD_res.plasma.value_zrt[:,:,-1])
pc1 = ax1.pcolormesh(1e3*CUPRAD_res.plasma.zgrid, 1e6*symmetric_y, symmetric_data.T, shading = 'auto')
ax1.set_ylim(-rlim,rlim)
cbar1 = fig.colorbar(pc1, ax=ax1, orientation = 'horizontal')


symmetric_y, symmetric_data =  mn.symmetrize_y(CUPRAD_res.rgrid,
                                    HHG.ComputeCutoff(
                                        Intensity_map[:,:,CUPRAD_res.Nt//2]/units.INTENSITYau,
                                        mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
                                        mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
                                    )[1]
                                )
pc2 = ax2.pcolormesh(1e3*CUPRAD_res.plasma.zgrid, 1e6*symmetric_y, symmetric_data.T, shading = 'auto')
levels = np.asarray([11,13,15,17,19,21,23])
contour = ax2.contour(1e3*CUPRAD_res.plasma.zgrid, 1e6*symmetric_y, symmetric_data.T, levels=levels, colors='black', linewidths=1.)

cbar2 = fig.colorbar(pc2, ax=ax2, orientation = 'horizontal')
cbar2.add_lines(contour)



ax1.set_ylabel(r'$\rho~[\mu \mathrm{m}]$')
ax1.set_xlabel(r'$z~[\mathrm{mm}]$')
ax2.set_xlabel(r'$z~[\mathrm{mm}]$')

cbar1.ax.set_xlabel('plasma density $[\mathrm{m}^{-3}]$')
cbar2.ax.set_xlabel('intensity [harmonic order]')

plt.show()


In [ ]:
image = pp.figure_driver()    
image.sf = [pp.plotter() for k2 in range(16)]

# image.sf[0].args = [CUPRAD_res.zgrid,np.max(CUPRAD_res.E_zrt[:,0,:],axis=1)]
# image.sf[1].args = [CUPRAD_res.zgrid,np.max(CUPRAD_res.E_zrt[:,:,:],axis=(1,2))]


image.sf[0].args = [CUPRAD_res.zgrid, HHG.ComputeCutoff(
                                        mn.FieldToIntensitySI(np.max(CUPRAD_res.E_zrt[:,0,:],axis=1))/units.INTENSITYau, omega0_au, Ip_au)[1]]

# np.max(CUPRAD_res.E_zrt[:,0,:],axis=1)]


# image.sf[1].args = [CUPRAD_res.zgrid,np.max(CUPRAD_res.E_zrt[:,:,:],axis=(1,2))]


# image.ylim_args = [-8500, 500]
pp.plot_preset(image)

In [ ]:
print(CUPRAD_res.omega0)

# program_path = os.environ['FSPA_PATH']+'/FSPA.e'

mydir = os.path.join('/mnt','d','git','MMA-interactive')
temporary_dir = 'temporary_FSPA_dir'
temporary_dir = os.path.join(mydir,temporary_dir)
if os.path.exists(temporary_dir): shutil.rmtree(temporary_dir)
os.makedirs(temporary_dir)

os.chdir(temporary_dir)

omega0_au = mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau')

A0_max = 1.2*(np.sqrt(CUPRAD_res.Intensity_entry/units.INTENSITYau))/omega0_au

print(A0_max)

A0_min = 0.1
dA = 0.01
# N_pts = 1000
N_pts = int(np.round((A0_max-A0_min)/dA)+1)
# omega0_au = mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau')
Ip_au = HHG.Ip_list[gas]
# print(HHG.Ip_list['Kr'])


print(A0_max,A0_min+N_pts*dA)
print(omega0_au,Ip_au)

with open('param.inp','w') as f:
    f.write('\n'.join([str(foo) for foo in [Ip_au,omega0_au,Horder,A0_min,dA,N_pts]]))
log_FSPA_run = subprocess.run(os.environ['FSPA_PATH']+'/FSPA.e',capture_output=True)


long_trajecotry_phase  = np.loadtxt('phase_long.dat')
short_trajecotry_phase  = np.loadtxt('phase_short.dat')

Igrid =  long_trajecotry_phase[:,0]
phase_long =  long_trajecotry_phase[:,7]
dI_phase_long = np.gradient(long_trajecotry_phase[:,7],Igrid,edge_order=2)

phase_short =  short_trajecotry_phase[:,7]
dI_phase_short = np.gradient(short_trajecotry_phase[:,7],Igrid,edge_order=2)


image = pp.figure_driver()    
image.sf = [pp.plotter() for k2 in range(16)]
image.sf[0].args = [Igrid,phase_long]
image.sf[1].args = [Igrid,phase_short]
# image.sf[0].args = [Igrid*units.INTENSITYau,phase_long]
# image.sf[1].args = [Igrid*units.INTENSITYau,phase_short]
# image.xlim_args = [Igrid[0]*units.INTENSITYau, 0.2e19]
# image.ylim_args = [-100, 20]
pp.plot_preset(image)


image = pp.figure_driver()    
image.sf = [pp.plotter() for k2 in range(16)]
# image.sf[0].args = [Igrid,dI_phase_long]
image.sf[1].args = [Igrid,dI_phase_short]

# image.sf[0].args = [Igrid*units.INTENSITYau,dI_phase_long]
# image.sf[1].args = [Igrid*units.INTENSITYau,dI_phase_short]

# image.ylim_args = [-8500, 500]
pp.plot_preset(image)



# shutil.rmtree(temporary_dir)

In [ ]:
from numpy.polynomial import Polynomial

image = pp.figure_driver()    
image.sf = [pp.plotter() for k2 in range(16)]
image.sf[0].args = [Igrid,dI_phase_short]
image.sf[1].args = [Igrid,dI_phase_long]
image.xlim_args = [(omega0_au*A0_min)**2, 0.002]
I_points = [0.0004,0.0005,0.0015,0.002]



print(mn.FindInterval(Igrid,I_points))

points_indexes = mn.FindInterval(Igrid,I_points)

print(dI_phase_short[mn.FindInterval(Igrid,I_points)])

p = Polynomial.fit(I_points,dI_phase_short[mn.FindInterval(Igrid,I_points)],3)

# print(p(0.0004))

image.sf[2].args = [Igrid,p(Igrid)]

dI_phase_short_new = np.copy(dI_phase_short)
dI_phase_short_new[points_indexes[0]:points_indexes[-1]] = p(Igrid[points_indexes[0]:points_indexes[-1]])


image.sf[3].args = [Igrid,dI_phase_short_new, '--']

pp.plot_preset(image)


In [ ]:
## Additional dephasings:

### SFA
alpha_values    = -np.interp(Intensity_map/units.INTENSITYau,Igrid,dI_phase_short_new) # a.u.
delta_k_dipole  =  Intensity_spatial_gradients * alpha_values/units.INTENSITYau        # SI


### XUV
delta_k_XUV     = CUPRAD_res.k0_wave*(nXUV-1.)

print(delta_k_XUV)


image = pp.figure_driver()    
image.sf = [pp.plotter() for k2 in range(16)]
image.sf[0].args = [CUPRAD_res.zgrid, phase_spatial_gradients[:,0,CUPRAD_res.Nt//2]]
pp.plot_preset(image)


image = pp.figure_driver()    
image.sf = [pp.plotter() for k2 in range(16)]
image.sf[0].args = [CUPRAD_res.zgrid, delta_k_dipole[:,0,CUPRAD_res.Nt//2]]
pp.plot_preset(image)


image = pp.figure_driver()    
image.sf = [pp.plotter() for k2 in range(16)]
image.sf[0].args = [CUPRAD_res.zgrid, Horder*(delta_k_dipole[:,0,CUPRAD_res.Nt//2] + CUPRAD_res.k0_wave*(nXUV-1.)) + delta_k_dipole[:,0,CUPRAD_res.Nt//2]]
image.sf[1].args = [CUPRAD_res.zgrid, Horder*(phase_spatial_gradients[:,0,CUPRAD_res.Nt//2] + CUPRAD_res.k0_wave*(nXUV-1.))]
image.sf[2].args = [CUPRAD_res.zgrid, delta_k_dipole[:,0,CUPRAD_res.Nt//2]]
pp.plot_preset(image)


## Plot the propagating pulse
We choose the time-and-space window to see the pulse as it propagates through the medium. Note that we measure the intensity by the "expected harmonic cutoff", these units are obtained by the formula $E_{\text{cut-off}} = I_P + 3.17U_p$ (it is directly proportional since $U_p$ is linearly proportional to the intensity). Then we plot the plasma channel create by the passage of the pulse. We show both absolute density of free electrons and also relative to the local density.

There are more technical details about the data: We plot the pulse directly as it is stored in the file. This means that we a co-moving frame defined by the group velocity, $v_g$, of the pulse: this is the computational window of CUPRAD. The group velocity $v_g$ is defined from the linear dispersion relation and depends on the chosen reference pressure and central wavelength. Possible density modulation is relative to this reference pressure, whcih is the reason we use the average pressure in our examples. Physically speaking, $v_g$ is arbitrary and needs to be considered in further processing. For example, the Pyrhonic class represented by `CUPRAD_res` contains methods to adjust to the reference given by the speed of light (both activelly by changing the data or just by sychronising the clocks in the $t$-grid).

In [ ]:

# tlim = np.asarray([t_plot_span*CUPRAD_res.pulse_duration_entry for t_plot_span in (-2. , 2.)])
# tlim2 = np.asarray([t_plot_span*CUPRAD_res2.pulse_duration_entry for t_plot_span in (-2. , 2.)])
# # rlim = 600

In [ ]:
# # Code to generate the animated figure

# k_t_min, k_t_max = mn.FindInterval(1e15*CUPRAD_res.tgrid,1.05*tlim)
# k_t_min2, k_t_max2 = mn.FindInterval(1e15*CUPRAD_res2.tgrid,1.05*tlim2)

# # k_r_max          = mn.FindInterval(1e6*CUPRAD_res.rgrid ,1.05*rlim)

# fig = plt.figure(figsize=(14, 5.5))


# ax1 = plt.subplot2grid((1, 2), (0, 0))  # Upper left
# ax2 = plt.subplot2grid((1, 2), (0, 1))  # Upper right

# # r_grid, sym_data = mn.symmetrize_y(1e6*CUPRAD_res.rgrid[:k_r_max],
# #                     (
# #                     HHG.ComputeCutoff(
# #                         mn.FieldToIntensitySI(CUPRAD_res.E_zrt[0,:k_r_max,k_t_min:k_t_max])/units.INTENSITYau,
# #                         mn.ConvertPhoton(CUPRAD_res.omega0,'omegaSI','omegaau'),
# #                         mn.ConvertPhoton(CUPRAD_res.Ip_eV,'eV','omegaau')
# #                     )[1]
# #                     ).T)

# pc1 = ax1.pcolormesh(1e15*CUPRAD_res.tgrid[k_t_min:k_t_max], 1e6*CUPRAD_res.rgrid, CUPRAD_res.E_zrt[0,:,k_t_min:k_t_max], shading='auto', cmap = 'seismic')
# pc2 = ax2.pcolormesh(1e15*CUPRAD_res2.tgrid[k_t_min2:k_t_max2], 1e6*CUPRAD_res2.rgrid, CUPRAD_res2.E_zrt[0,:,k_t_min2:k_t_max2], shading='auto', cmap = 'seismic')

# ax1.set_xlim(tlim)
# # ax1.set_ylim((-rlim,rlim))

# ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[0]) + ' mm')
# ax1.set_xlabel(r'$t~[\mathrm{fs}]$')
# ax1.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

# cbar1 = fig.colorbar(pc1, ax=ax1)
# cbar2 = fig.colorbar(pc2, ax=ax2)
# # cbar.ax.set_ylabel(r'Intensity [harmonic cut-off]', rotation=90)




# def update(frame):
#     # Update the data
#     data1 = CUPRAD_res.E_zrt[frame,:,k_t_min:k_t_max]
#     data2 = CUPRAD_res2.E_zrt[frame,:,k_t_min2:k_t_max2]
    
#     # Update the colors
#     pc1.set_array(data1.ravel())
#     pc1.set_clim(data1.min(), data1.max())
#     cbar1.update_normal(pc1)

#     pc2.set_array(data2.ravel())
#     pc2.set_clim(data2.min(), data2.max())
#     cbar2.update_normal(pc2)

#     ax1.set_title("z={:.2f}".format(1e3*CUPRAD_res.zgrid[frame]) + ' mm')

#     # Update the progress indicator
#     # progress_line.set_data([1e3*CUPRAD_res.zgrid[frame], 1e3*CUPRAD_res.zgrid[frame]],
#     #                         [CUPRAD_res.density_mod_profile_mbar.min(), CUPRAD_res.density_mod_profile_mbar.max()])

#     return [pc1,pc2]

# # Ensure the layout does not have overlaps and everything is nicely spaced
# fig.tight_layout() 

# ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(CUPRAD_res.zgrid), blit=True)

# HTML(ani.to_jshtml())